In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import mixed_precision
from sklearn.metrics import confusion_matrix, classification_report 
from transformers import BertTokenizer
import pandas as pd
import zipfile
import os 
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample



c:\Users\makri\.conda\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\makri\.conda\envs\tf\lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:

# Reading datasets
with zipfile.ZipFile('llm_train_df.csv.zip','r') as zip:
    with zip.open('llm_train_df.csv') as f:
        llm_data = pd.read_csv(f,encoding='ISO-8859-1')

with zipfile.ZipFile('combined_train_df_2.csv.zip','r') as zip:
    with zip.open('combined_train_df_2.csv') as f:
        train = pd.read_csv(f,encoding='ISO-8859-1')

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    try:
        # Enable memory growth for the first (and only) GPU
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print(f"Memory growth enabled for {gpus[0]}")
    except RuntimeError as e:
        print(e)  # This happens if GPUs are initialized before setting memory growth
else:
    print("No GPU found. Running on CPU.")

Memory growth enabled for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [4]:
llm_data['Human_vs_AI'] = llm_data.apply(lambda x: 0 if x['type']=='main' else 1,axis=1)
train['Human_vs_AI'] = 0
combined = pd.concat([llm_data,train],axis=0)

# Remove LIAR 2 middle category
combined = combined[combined['label']!=3]

In [5]:
len(combined)

1504705

In [ ]:
# Targets distributions

display(combined['binary_label'].value_counts(normalize=True))
display(combined['Human_vs_AI'].value_counts(normalize=True))

binary_label
0    0.525513
1    0.474487
Name: proportion, dtype: float64

Human_vs_AI
0    0.814591
1    0.185409
Name: proportion, dtype: float64

In [ ]:
# Oversampling minority

minority = combined[combined['Human_vs_AI'] == 1]
majority = combined[combined['Human_vs_AI'] == 0]


minority_upsampled = resample(minority, 
                              replace=True,    # allow duplicates
                              n_samples=800000, # match majority count
                              random_state=42) # reproducible


combined_balanced = pd.concat([majority, minority_upsampled])

display(combined_balanced['Human_vs_AI'].value_counts(normalize=True))
display(combined_balanced['binary_label'].value_counts(normalize=True))


Human_vs_AI
0    0.605078
1    0.394922
Name: proportion, dtype: float64

binary_label
0    0.638499
1    0.361501
Name: proportion, dtype: float64

In [8]:
len(combined_balanced)

2025719

In [ ]:
BATCH_SIZE = 16
SEED = 42


# Test-Validation Split
train, val = train_test_split(combined_balanced,test_size=0.3,random_state=42)#stratify=temp_train['dataset'])



# Training data
X_train = train['text'].values  
y_train = {
    'Fake_News_Output': train['binary_label'].values,
    'Human_vs_AI': train['Human_vs_AI'].values
} 

# Validation data
X_val = val['text'].values
val_train = {
    'Fake_News_Output': val['binary_label'].values,
    'Human_vs_AI': val['Human_vs_AI'].values
} 



In [8]:
X_train = [str(x) for x in X_train]
X_val = [str(x) for x in X_val]

In [ ]:
# Train and validation  distributions by dataset
display(val['dataset'].value_counts(normalize=True))

display(train['dataset'].value_counts(normalize=True))



dataset
LLM News Dataset                  0.445919
Fakeddit                          0.347621
Kaggle 1 - Fake News              0.146104
Kaggle 2 - News Project           0.040266
Kaggle 3 - Fake News Detection    0.012284
LIAR 2                            0.007806
Name: proportion, dtype: float64

dataset
LLM News Dataset                  0.445887
Fakeddit                          0.347203
Kaggle 1 - Fake News              0.146772
Kaggle 2 - News Project           0.040089
Kaggle 3 - Fake News Detection    0.012199
LIAR 2                            0.007850
Name: proportion, dtype: float64

In [ ]:
# Tokenization

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(X_train, truncation=True, padding = 'max_length',max_length=60, return_tensors="tf")

val_encodings = tokenizer(
    X_val,
    truncation=True,
    padding = 'max_length',
    max_length = 60,
    return_tensors="tf"
)

c:\Users\makri\.conda\envs\tf\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# Prepare dataset
inputs = {
    'input_word_ids': train_encodings['input_ids'],
    'input_mask': train_encodings['attention_mask'],
    'input_type_ids': train_encodings['token_type_ids']
}


val_inputs = {
    'input_word_ids': val_encodings['input_ids'],
    'input_mask': val_encodings['attention_mask'],
    'input_type_ids': val_encodings['token_type_ids']
}


# Now build dataset properly
train_ds = tf.data.Dataset.from_tensor_slices((inputs,  y_train)).shuffle(buffer_size=len(X_train),seed=SEED).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)
val_ds = tf.data.Dataset.from_tensor_slices((val_inputs, val_train))\
         .batch(BATCH_SIZE)\
         .prefetch(tf.data.AUTOTUNE)


In [12]:
# Build Model
mixed_precision.set_global_policy('mixed_float16')

# BERT encoder 
bert_model = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3",
    trainable=True
)


# Inputs
input_ids = tf.keras.Input(shape=(60,), dtype=tf.int32, name="input_word_ids")
input_mask = tf.keras.Input(shape=(60,), dtype=tf.int32, name="input_mask")
type_ids = tf.keras.Input(shape=(60), dtype=tf.int32, name="input_type_ids")

bert_inputs = {
    'input_word_ids': input_ids,
    'input_mask': input_mask,
    'input_type_ids': type_ids
}

bert_outputs = bert_model(bert_inputs)
cls_token = bert_outputs['pooled_output']

x = tf.keras.layers.Dropout(0.1)(cls_token)

# First classification head
fake_news = tf.keras.layers.Dense(1,activation='sigmoid',name='Fake_News_Output',dtype="float32")(x)

# Second classification head
human_ai = tf.keras.layers.Dense(1, activation='sigmoid', name='Human_vs_AI',dtype='float32')(x)

model = tf.keras.Model(inputs=[input_ids, input_mask, type_ids], outputs=[fake_news,human_ai])
model.summary()


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4050 Laptop GPU, compute capability 8.9
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_mask (InputLayer)        [(None, 60)]         0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 60)]         0           []                               
                                                                                                  
 input_word_ids (InputLayer)    [(None, 60)]         0           []                               
                                        

In [13]:
model.compile(loss = {'Fake_News_Output':'binary_crossentropy', 
                      'Human_vs_AI': 'binary_crossentropy'}, 
                      optimizer = tf.keras.optimizers.Adam(2e-5),  
                      metrics={'Fake_News_Output': 'accuracy',
                               'Human_vs_AI': 'accuracy'
    })

In [14]:
history = model.fit(train_ds, validation_data=val_ds, epochs=1)

c:\Users\makri\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/StatefulPartitionedCall:1", shape=(None,), dtype=int32), values=Tensor("mul_4:0", dtype=float32), dense_shape=Tensor("Adam/gradients/StatefulPartitionedCall:2", shape=(None,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


88626/88626 [==============================] - 16991s 192ms/step - loss: 0.2831 - Fake_News_Output_loss: 0.2241 - Human_vs_AI_loss: 0.0590 - Fake_News_Output_accuracy: 0.9047 - Human_vs_AI_accuracy: 0.9779 - val_loss: 0.2628 - val_Fake_News_Output_loss: 0.2050 - val_Human_vs_AI_loss: 0.0578 - val_Fake_News_Output_accuracy: 0.9179 - val_Human_vs_AI_accuracy: 0.9810


# Testing #

In [26]:
# Test data 
with zipfile.ZipFile('llm_test_df.csv.zip','r') as zip:
    with zip.open('llm_test_df.csv') as f:
        llm_data_test = pd.read_csv(f,encoding='ISO-8859-1')

with zipfile.ZipFile('test_df_2.csv.zip','r') as zip:
    with zip.open('test_df_2.csv') as f:
        test = pd.read_csv(f,encoding='ISO-8859-1')

In [27]:
llm_data_test['Human_vs_AI'] = llm_data_test.apply(lambda x: 0 if x['type']=='main' else 1,axis=1)
test['Human_vs_AI'] = 0
combined_test = pd.concat([llm_data_test,test],axis=0)

# Remove LIAR 2 middle category
combined_test = combined_test[combined_test['label']!=3]

In [ ]:
# Targets distributions
display(combined_test['binary_label'].value_counts(normalize=True))
display(combined_test['Human_vs_AI'].value_counts(normalize=True))

binary_label
0    0.667293
1    0.332707
Name: proportion, dtype: float64

Human_vs_AI
1    0.60696
0    0.39304
Name: proportion, dtype: float64

In [ ]:
# Distribution by dataset
display(combined_test['dataset'].value_counts(normalize=True))

dataset
LLM News Dataset                  0.831610
Kaggle 2 - News Project           0.034809
Kaggle 3 - Fake News Detection    0.034809
Fakeddit                          0.034809
Kaggle 1 - Fake News              0.034809
LIAR 2                            0.029153
Name: proportion, dtype: float64

In [ ]:
# Random 4000 samples from LLM dataset

minority = combined_test[combined_test['dataset']!= 'LLM News Dataset']
majority = combined_test[combined_test['dataset'] == 'LLM News Dataset']



majority_downsampled = resample(majority, 
                              replace=True,    # allow duplicates
                              n_samples=4000, # match majority count
                              random_state=42) # reproducible


combined_test_balanced = pd.concat([minority, majority_downsampled])

display(combined_test_balanced['Human_vs_AI'].value_counts(normalize=True))
display(combined_test_balanced['binary_label'].value_counts(normalize=True))

display(combined_test_balanced['dataset'].value_counts(normalize=True))

Human_vs_AI
0    0.876317
1    0.123683
Name: proportion, dtype: float64

binary_label
0    0.519229
1    0.480771
Name: proportion, dtype: float64

dataset
Kaggle 2 - News Project           0.171306
Kaggle 3 - Fake News Detection    0.171306
Fakeddit                          0.171306
Kaggle 1 - Fake News              0.171306
LLM News Dataset                  0.171306
LIAR 2                            0.143469
Name: proportion, dtype: float64

In [31]:
X_test = combined_test_balanced['text'].values
y_test = {
    'Fake_News_Output': combined_test_balanced['binary_label'].values,
    'Human_vs_AI': combined_test_balanced['Human_vs_AI'].values
} 

X_test = [str(x) for x in X_test]

In [32]:
test_encodings = tokenizer(X_test, truncation=True, padding = 'max_length',max_length=60, return_tensors="tf")

In [33]:
# Prepare dataset
inputs_test = {
    'input_word_ids': test_encodings['input_ids'],
    'input_mask': test_encodings['attention_mask'],
    'input_type_ids': test_encodings['token_type_ids']
}

# Predictions fake news
predictions = model.predict(dict(inputs_test))

threshold = 0.5
preds = (predictions[0]>threshold).astype(int)
print(classification_report(preds,y_test["Fake_News_Output"]))

# Predictions Human-vs-AI
preds_new = (predictions[1]>threshold).astype(int)
print(classification_report(preds_new,y_test["Human_vs_AI"]))


730/730 [==============================] - 61s 82ms/step
              precision    recall  f1-score   support

           0       0.88      0.83      0.86     12741
           1       0.81      0.86      0.84     10609

    accuracy                           0.85     23350
   macro avg       0.84      0.85      0.85     23350
weighted avg       0.85      0.85      0.85     23350

              precision    recall  f1-score   support

           0       0.95      1.00      0.97     19501
           1       0.99      0.74      0.85      3849

    accuracy                           0.96     23350
   macro avg       0.97      0.87      0.91     23350
weighted avg       0.96      0.96      0.95     23350



In [34]:
# Per model accuracy (Fake news detection)
preds_new_new = pd.DataFrame(preds,index=combined_test_balanced.index)
concat = pd.concat([combined_test_balanced,preds_new_new],axis=1)

concat.columns.values[-1] = 'preds'

display(concat['preds'].value_counts())

accuracy_df = (concat['preds'] == concat['binary_label']).groupby(concat['dataset']).mean()

print(accuracy_df)


# Per model accuracy (Human-vs-AI)
preds_new_new = pd.DataFrame(preds_new,index=combined_test_balanced.index)
concat = pd.concat([combined_test_balanced,preds_new_new],axis=1)

concat.columns.values[-1] = 'preds'

display(concat['preds'].value_counts())

accuracy_df = (concat['preds'] == concat['Human_vs_AI']).groupby(concat['dataset']).mean()

print(accuracy_df)



preds
0    12741
1    10609
Name: count, dtype: int64

dataset
Fakeddit                          0.858750
Kaggle 1 - Fake News              0.948250
Kaggle 2 - News Project           0.784500
Kaggle 3 - Fake News Detection    0.815750
LIAR 2                            0.706567
LLM News Dataset                  0.940250
dtype: float64


preds
0    19501
1     3849
Name: count, dtype: int64

dataset
Fakeddit                          0.998500
Kaggle 1 - Fake News              0.975000
Kaggle 2 - News Project           0.941000
Kaggle 3 - Fake News Detection    0.902500
LIAR 2                            0.975522
LLM News Dataset                  0.944750
dtype: float64


# Now test only on LLM data #

In [35]:
display(llm_data_test['binary_label'].value_counts(normalize=True))
display(llm_data_test['Human_vs_AI'].value_counts(normalize=True))

binary_label
0    0.704747
1    0.295253
Name: proportion, dtype: float64

Human_vs_AI
1    0.729861
0    0.270139
Name: proportion, dtype: float64

In [36]:
# Display models 
display(llm_data_test['model'].value_counts(normalize=True))

model
GPT3.5        0.263590
Mistral 7b    0.253345
Llama2 7b     0.242966
Llama2 13b    0.240100
Name: proportion, dtype: float64

In [37]:
X_test = llm_data_test['text'].values
y_test = {
    'Fake_News_Output': llm_data_test['binary_label'].values,
    'Human_vs_AI': llm_data_test['Human_vs_AI'].values
} 

X_test = [str(x) for x in X_test]

In [38]:
test_encodings = tokenizer(X_test, truncation=True, padding = 'max_length',max_length=60, return_tensors="tf")

In [39]:
# Prepare dataset
inputs_test = {
    'input_word_ids': test_encodings['input_ids'],
    'input_mask': test_encodings['attention_mask'],
    'input_type_ids': test_encodings['token_type_ids']
}

# Predictions Fake News
predictions = model.predict(dict(inputs_test))

threshold = 0.5
preds = (predictions[0]>threshold).astype(int)

print(classification_report(preds,y_test["Fake_News_Output"]))

# Predictions Human vs AI
preds_new = (predictions[1]>threshold).astype(int)

print(classification_report(preds_new,y_test["Human_vs_AI"]))

2987/2987 [==============================] - 252s 85ms/step
              precision    recall  f1-score   support

           0       0.99      0.93      0.96     71382
           1       0.83      0.97      0.89     24180

    accuracy                           0.94     95562
   macro avg       0.91      0.95      0.93     95562
weighted avg       0.95      0.94      0.94     95562

              precision    recall  f1-score   support

           0       0.83      0.96      0.89     22327
           1       0.99      0.94      0.96     73235

    accuracy                           0.94     95562
   macro avg       0.91      0.95      0.93     95562
weighted avg       0.95      0.94      0.95     95562



In [40]:
# Per model accuracy (Fake News Detection)
preds_new_new = pd.DataFrame(preds,index=llm_data_test.index)
concat = pd.concat([llm_data_test,preds_new_new],axis=1)

concat.columns.values[-1] = 'preds'

display(concat['preds'].value_counts())

accuracy_df = (concat['preds'] == concat['binary_label']).groupby(concat['model']).mean()

print(accuracy_df)


# Per model  accuracy (Human_vs_AI)
preds_new_new = pd.DataFrame(preds_new,index=llm_data_test.index)
concat = pd.concat([llm_data_test,preds_new_new],axis=1)

concat.columns.values[-1] = 'preds'

display(concat['preds'].value_counts())

accuracy_df = (concat['preds'] == concat['Human_vs_AI']).groupby(concat['model']).mean()

print(accuracy_df)




preds
0    71382
1    24180
Name: count, dtype: int64

model
GPT3.5        0.996676
Llama2 13b    0.981447
Llama2 7b     0.983744
Mistral 7b    0.992146
dtype: float64


preds
1    73235
0    22327
Name: count, dtype: int64

model
GPT3.5        0.996113
Llama2 13b    0.980705
Llama2 7b     0.982460
Mistral 7b    0.990681
dtype: float64
